In [2]:
import torch

import pathpyG as pp

print('Running on', pp.config['torch']['device'])

Running on cpu


In [3]:
# Put this as his in conftest as 'simple_paths_centralities'
paths = pp.WalkData()
paths.add(torch.tensor([[2, 1, 3], [1, 3, 5]]))  
paths.add(torch.tensor([[0, 1], [1, 3]]))  
paths.add(torch.tensor([[3], [4]]))

simple_paths_centralities = paths

In [4]:
# paths = pp.PathData()
# paths.add_walk(torch.tensor([[0,2,3],[2,3,4]]),freq=3) # A -> C -> D
# paths.add_walk(torch.tensor([[0,2],[2,3]])) # A -> C -> D
# paths.add_walk(torch.tensor([[1,2],[2,4]])) # B -> C -> E
# paths.add_walk(torch.tensor([[4],[5]]))
# paths.add_walk(torch.tensor([[1,2],[2,4]])) # B -> C -> E


In [5]:
index, edge_weights = paths.edge_index_k_weighted(k=2)
index, edge_weights

(tensor([[[0, 1],
          [1, 3],
          [2, 1]],
 
         [[1, 3],
          [3, 5],
          [1, 3]]]),
 tensor([1., 1., 1.]))

In [6]:
index, edge_weights = paths.edge_index_k_weighted(k=1)

In [7]:
from collections import defaultdict

def node_traversals(paths):
    """Calculates the number of times any path traverses each of the nodes.

    Parameters
    ----------
    paths: Paths

    Returns
    -------
    dict
    """
    traversals = defaultdict(lambda: 0)
    for path_id, path_edgelist in paths.paths.items():
        path_seq = paths.walk_to_node_seq(path_edgelist)
        for node in path_seq:
            traversals[node.item()] += paths.path_freq[path_id]
    return traversals

In [10]:
from pathpyG.algorithms.centrality import node_traversals

In [11]:
node_traversals(paths)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [13]:
from pathpyG.algorithms.centrality import visitation_probabilities

In [14]:
def test_visitation_probabilities(simple_paths_centralities):
    traversals_dict = visitation_probabilities(simple_paths_centralities)
    assert set(traversals_dict.keys()) == {0,1,2,3,4,5}
    assert traversals_dict[0] == 1/9
    assert traversals_dict[1] == 2/9
    assert traversals_dict[2] == 1/9
    assert traversals_dict[3] == 3/9
    assert traversals_dict[4] == 1/9
    assert traversals_dict[5] == 1/9

test_visitation_probabilities(simple_paths_centralities)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [339]:
test_shortest_paths(paths)

IndexError occurred. Reached maximum path length of 4


In [340]:
# @betweenness.register(Paths)
def betweenness(paths, normalized=False):
    """Calculates the betweenness of nodes based on observed shortest paths
    between all pairs of nodes

    Parameters
    ----------
    paths:
        Paths object
    normalized: bool
        normalize such that largest value is 1.0

    Returns
    -------
    dict
    """
    assert isinstance(paths, pp.PathData), "argument must be an instance of pathpy.Paths"
    node_centralities = defaultdict(lambda: 0)

    # Log.add('Calculating betweenness in paths ...', Severity.INFO)

    all_paths = shortest_paths(paths)

    for s in all_paths:
        for d in all_paths[s]:
            for p in all_paths[s][d]:
                for x in p[1:-1]:
                    if s != d != x:
                        node_centralities[x.item()] += 1.0 / len(all_paths[s][d])
    if normalized:
        max_centr = max(node_centralities.values())
        for v in node_centralities:
            node_centralities[v] /= max_centr
    # assign zero values to nodes not occurring on shortest paths
    nodes = [v.item() for v in paths.edge_index.reshape(-1).unique(dim=0)]
    for v in nodes:
        node_centralities[v] += 0
    # Log.add('finished.')
    return node_centralities

betweenness(paths,normalized=False)

IndexError occurred. Reached maximum path length of 4


defaultdict(<function __main__.betweenness.<locals>.<lambda>()>,
            {1: 3.0, 3: 2.0, 0: 0, 2: 0, 4: 0, 5: 0})

In [346]:
def test_betweenness_paths(simple_paths_centralities):
    bw = betweenness(simple_paths_centralities,normalized=False)
    # 1 is in the shortest path between 0-5,2-3,2-5
    assert bw[1] == 3.0
    # 1 is in the shortest path between 2-5,1-5
    assert bw[3] == 2.0

test_betweenness_paths(paths)

IndexError occurred. Reached maximum path length of 4


In [347]:

def distance_matrix(paths):
    """
    Calculates shortest path distances between all pairs of
    nodes based on the observed shortest paths (and subpaths)
    """
    dist = defaultdict(lambda: defaultdict(lambda: _np.inf))
    # Log.add('Calculating distance matrix based on empirical paths ...', Severity.INFO)
    nodes = [v.item() for v in paths.edge_index.reshape(-1).unique(dim=0)] # NOTE: modify once set of nodes can be obtained from path obeject
    for v in nodes:
        dist[v][v] = 0

    p_length = 1
    index, edge_weights = paths.edge_index_k_weighted(k=p_length)
    sources = index[0]
    destinations = index[-1]
    for e, (s, d) in enumerate(zip(sources, destinations)):
        s = s.item()
        d = d.item()
        dist[s][d] = p_length
        # s_p[s][d] = set({torch.tensor([s,d])})
    p_length += 1
    while True: # until max path length
        try:
            index, edge_weights = paths.edge_index_k_weighted(k=p_length)
            sources = index[0, :, 0]
            destinations = index[1, :, -1]
            for e, (s, d) in enumerate(zip(sources, destinations)):
                s = s.item()
                d = d.item()
                if p_length < dist[s][d]:
                    # update shortest path length
                    dist[s][d] = p_length
            p_length += 1
        except IndexError:
            print(f"IndexError occurred. Reached maximum path length of {p_length}")
            break
    return dist
distance_matrix(paths)
    

IndexError occurred. Reached maximum path length of 4


defaultdict(<function __main__.distance_matrix.<locals>.<lambda>()>,
            {0: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {0: 0, 1: 1, 3: 2}),
             1: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {1: 0, 3: 1, 5: 2}),
             2: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {2: 0, 1: 1, 3: 2, 5: 3}),
             3: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {3: 0, 4: 1, 5: 1}),
             4: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {4: 0}),
             5: defaultdict(<function __main__.distance_matrix.<locals>.<lambda>.<locals>.<lambda>()>,
                         {5: 0})})

In [352]:
def test_distance_matrix_paths(simple_paths_centralities):
    dm = distance_matrix(simple_paths_centralities)
    assert dm[0] == {0: 0, 1: 1, 3: 2}
    assert dm[1] == {1: 0, 3: 1, 5: 2}
    assert dm[2] == {2: 0, 1: 1, 3: 2, 5: 3}
    assert dm[3] == {3: 0, 4: 1, 5: 1}
    assert dm[4] == {4: 0}
    assert dm[5] == {5: 0}

test_distance_matrix_paths(paths)

IndexError occurred. Reached maximum path length of 4


In [355]:
def closeness(paths, normalized=False):
    """Calculates the closeness of nodes based on observed shortest paths
    between all nodes

    Parameters
    ----------
    paths: Paths
    normalized: bool
        normalize such that largest value is 1.0

    Returns
    -------
    dict
    """
    node_centralities = defaultdict(lambda: 0)
    distances = distance_matrix(paths)
    nodes = [v.item() for v in paths.edge_index.reshape(-1).unique(dim=0)] # NOTE: modify once set of nodes can be obtained from path obeject

    for x in nodes:
        # calculate closeness centrality of x
        for d in nodes:
            if x != d and distances[d][x] < _np.inf:
                node_centralities[x] += 1.0 / distances[d][x]

    # assign zero values to nodes not occurring
    
    for v in nodes:
        node_centralities[v] += 0.0

    if normalized:
        m = max(node_centralities.values())
        for v in nodes:
            node_centralities[v] /= m

    return node_centralities
closeness(paths, normalized=False)

IndexError occurred. Reached maximum path length of 4


defaultdict(<function __main__.closeness.<locals>.<lambda>()>,
            {1: 2.0, 3: 2.0, 4: 1.0, 5: 1.8333333333333333, 0: 0.0, 2: 0.0})

In [360]:
def test_closeness_paths(simple_paths_centralities):
    c = closeness(simple_paths_centralities, normalized=False)
    assert c[0] == 0.0
    # 1 reachable from 0 and 2 in one step
    assert c[1] == 1/1 + 1/1
    assert c[2] == 0
    # 3 reachable from 1 in one step, from 0 and 3 in two steps
    assert c[3] == 1 + 1/2 + 1/2
    assert c[4] == 1
    # 5 reachable from 3 in one step, from 1 in two steps, from 2 in three steps
    assert c[5] == 1 + 1/2 + 1/3
test_closeness_paths(paths)

IndexError occurred. Reached maximum path length of 4
